# Pose-Based Form Checker + Muscle Growth Predictor

Uses **pre-recorded exercise videos** to generate reference animations.
User performs in front of webcam → live feedback via stick figures (blue = reference, green/red = user).
Streamed to phone via Flask.

After the workout, users input workout data → predicts muscle growth using **bulk.ipynb** model.

**Folder structure:**
```
project/
├── recordFromVideo.ipynb
├── 32Fit/
│   ├── bulk.ipynb
│   └── mostCommonExercises.csv
└── exercises/
    ├── squat.mp4
    ├── pushup.mp4
    └── dumbbell_curls.mp4
```

In [8]:
import cv2
import mediapipe as mp
import numpy as np
import pickle
import time
import os
import glob
from flask import Flask, Response, render_template_string, request, jsonify
import threading
import requests
import json

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Global frame for streaming
current_frame = None

# Bulk predictor API endpoint (running bulk.ipynb Flask server)
BULK_API_URL = "http://localhost:3001/bulk"

# Exercise queue management
exercise_queue = []
current_exercise_index = 0
is_exercising = False
exercise_active = False
stop_exercise_flag = False

## Helper: Calculate Angle

In [9]:
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    ab = a - b
    bc = c - b
    cosine_angle = np.dot(ab, bc) / (np.linalg.norm(ab) * np.linalg.norm(bc) + 1e-6)
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)

## Extract Reference from Video

In [10]:
def extract_reference_from_video(video_path: str, exercise_name: str, target_fps: float = 10.0):
    if not os.path.exists(video_path):
        print(f"Video not found: {video_path}")
        return

    pose = mp_pose.Pose(static_image_mode=False,
                        model_complexity=1,
                        enable_segmentation=False,
                        min_detection_confidence=0.5,
                        min_tracking_confidence=0.5)

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Cannot open video: {video_path}")
        return

    video_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_interval = max(1, int(video_fps / target_fps))
    references = []
    frame_count = 0

    print(f"Extracting reference from {video_path} @ ~{target_fps} FPS...")

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % frame_interval == 0:
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(rgb)

            if results.pose_landmarks:
                lm_list = [(lm.x, lm.y, lm.z, lm.visibility) for lm in results.pose_landmarks.landmark]
                references.append(lm_list)

        frame_count += 1

    cap.release()
    pose.close()

    pkl_path = f"{exercise_name}.pkl"
    with open(pkl_path, 'wb') as f:
        pickle.dump(references, f)

    print(f"Saved {len(references)} reference frames → {pkl_path}")

## Auto-Discover & Extract All Videos

In [11]:
def extract_all_references_from_folder(folder='exercises', target_fps=10.0):
    if not os.path.exists(folder):
        print(f"Folder '{folder}' not found!")
        return

    video_extensions = ['*.mp4', '*.avi', '*.mov', '*.mkv', '*.wmv']
    video_files = []
    for ext in video_extensions:
        video_files.extend(glob.glob(os.path.join(folder, ext)))

    if not video_files:
        print("No videos found in exercises/ folder.")
        return

    print(f"Found {len(video_files)} video(s). Extracting references...\n")
    for video_path in video_files:
        name = os.path.splitext(os.path.basename(video_path))[0]
        extract_reference_from_video(video_path, name, target_fps)
    print("\nAll references extracted!")

## Run Once: Extract All References

In [12]:
extract_all_references_from_folder()

Folder 'exercises' not found!


## Perform Exercise + Predict Growth

In [13]:
def perform_exercise_and_predict(exercise_name):
    global current_frame, stop_exercise_flag, exercise_active
    
    pkl_path = f"{exercise_name}.pkl"
    if not os.path.exists(pkl_path):
        print(f"Reference not found: {pkl_path}. Run extraction first!")
        return

    with open(pkl_path, 'rb') as f:
        reference_sequence = pickle.load(f)
    
    if not reference_sequence:
        print("Empty reference sequence.")
        return

    pose = mp_pose.Pose()
    cap = cv2.VideoCapture(0)
    ref_index = 0
    animation_fps = 10
    frame_delay = 1.0 / animation_fps
    last_ref_time = time.time()
    angle_threshold = 20
    
    pose_connections = mp_pose.POSE_CONNECTIONS
    
    print(f"Starting {exercise_name}. Match the BLUE stick figure.")
    print(f"Stream: http://[YOUR-PC-IP]:5000  |  Stop from web interface.\n")
    
    exercise_active = True
    stop_exercise_flag = False

    while cap.isOpened() and not stop_exercise_flag:
        ret, frame = cap.read()
        if not ret:
            break
        
        height, width, _ = frame.shape
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb)
        
        black = np.zeros((height, width, 3), dtype=np.uint8)
        
        # Draw reference (BLUE)
        ref_lm = reference_sequence[ref_index]
        for conn in pose_connections:
            start, end = ref_lm[conn[0]], ref_lm[conn[1]]
            if start[3] > 0.1 and end[3] > 0.1:
                cv2.line(black,
                         (int(start[0] * width), int(start[1] * height)),
                         (int(end[0] * width), int(end[1] * height)),
                         (255, 0, 0), 2)
        
        # Draw user (GREEN if correct, RED if wrong)
        if results.pose_landmarks:
            user_lm = [(lm.x, lm.y, lm.z, lm.visibility) for lm in results.pose_landmarks.landmark]
            
            for conn in pose_connections:
                i1, i2 = conn
                r1, r2 = ref_lm[i1], ref_lm[i2]
                u1, u2 = user_lm[i1], user_lm[i2]
                
                if all(x[3] > 0.1 for x in [r1, r2, u1, u2]):
                    parent_idx = None
                    for c in pose_connections:
                        if c[1] == i2 and c[0] != i1:
                            parent_idx = c[0]; break
                        if c[0] == i2 and c[1] != i1:
                            parent_idx = c[1]; break
                    
                    if parent_idx is not None and ref_lm[parent_idx][3] > 0.1 and user_lm[parent_idx][3] > 0.1:
                        rp, up = ref_lm[parent_idx], user_lm[parent_idx]
                        
                        ref_angle = calculate_angle((r1[0], r1[1]), (r2[0], r2[1]), (rp[0], rp[1]))
                        user_angle = calculate_angle((u1[0], u1[1]), (u2[0], u2[1]), (up[0], up[1]))
                        
                        color = (0, 255, 0) if abs(ref_angle - user_angle) < angle_threshold else (0, 0, 255)
                        
                        cv2.line(black,
                                 (int(u1[0] * width), int(u1[1] * height)),
                                 (int(u2[0] * width), int(u2[1] * height)),
                                 color, 2)
        
        current_frame = cv2.imencode('.jpg', black)[1].tobytes()
        
        if time.time() - last_ref_time >= frame_delay:
            last_ref_time = time.time()
            ref_index = (ref_index + 1) % len(reference_sequence)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()
    exercise_active = False
    print(f"Exercise {exercise_name} completed!")


def get_available_exercises():
    """Get list of available exercises from .pkl files"""
    pkl_files = glob.glob("*.pkl")
    return [os.path.splitext(f)[0] for f in pkl_files]

## Start Streaming Server

In [14]:
app = Flask(__name__)

HTML_TEMPLATE = '''
<!DOCTYPE html>
<html>
<head>
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Workout Trainer</title>
    <style>
        * { margin: 0; padding: 0; box-sizing: border-box; }
        body {
            font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            min-height: 100vh;
            padding: 20px;
        }
        .container {
            max-width: 900px;
            margin: 0 auto;
            background: white;
            border-radius: 20px;
            padding: 20px;
            box-shadow: 0 20px 60px rgba(0,0,0,0.3);
        }
        h1 { color: #667eea; margin-bottom: 20px; text-align: center; }
        h2 { color: #333; margin: 20px 0 10px; font-size: 1.3em; }
        .video-container {
            position: relative;
            width: 100%;
            background: #000;
            border-radius: 10px;
            overflow: hidden;
            margin-bottom: 20px;
        }
        .video-container img { width: 100%; display: block; }
        .current-exercise {
            background: #f0f4ff;
            padding: 15px;
            border-radius: 10px;
            margin-bottom: 20px;
            text-align: center;
        }
        .current-exercise h3 { color: #667eea; font-size: 1.5em; }
        .queue-section {
            background: #f9fafb;
            padding: 15px;
            border-radius: 10px;
            margin-bottom: 20px;
        }
        .exercise-list { margin: 10px 0; }
        .exercise-item {
            background: white;
            padding: 10px;
            margin: 5px 0;
            border-radius: 8px;
            display: grid;
            grid-template-columns: 1fr auto;
            gap: 8px;
            align-items: center;
        }
        .exercise-item .meta { font-size: 0.9em; color: #374151; }
        .exercise-item.active { background: #667eea; color: white; font-weight: bold; }
        .exercise-item.active .meta { color: #e0e7ff; }
        select, input, button {
            width: 100%;
            padding: 12px;
            margin: 8px 0;
            border: 2px solid #e5e7eb;
            border-radius: 8px;
            font-size: 16px;
        }
        button {
            background: #667eea;
            color: white;
            border: none;
            cursor: pointer;
            font-weight: bold;
            transition: all 0.3s;
        }
        button:hover { background: #5568d3; transform: translateY(-2px); }
        button:active { transform: translateY(0); }
        .btn-danger { background: #ef4444; }
        .btn-danger:hover { background: #dc2626; }
        .btn-success { background: #10b981; }
        .btn-success:hover { background: #059669; }
        .btn-secondary { background: #6b7280; }
        .btn-secondary:hover { background: #4b5563; }
        .form-grid {
            display: grid;
            grid-template-columns: 1fr 1fr;
            gap: 10px;
        }
        .form-grid-3 {
            display: grid;
            grid-template-columns: repeat(3, 1fr);
            gap: 10px;
        }
        .form-grid input, .form-grid select, .form-grid-3 input, .form-grid-3 select { margin: 0; }
        .full-width { grid-column: 1 / -1; }
        .prediction-result {
            background: #f0fdf4;
            border: 2px solid #10b981;
            padding: 15px;
            border-radius: 10px;
            margin-top: 15px;
        }
        .prediction-result h3 { color: #10b981; margin-bottom: 10px; }
        .muscle-growth { margin: 10px 0; padding: 10px; background: white; border-radius: 5px; }
        .hidden { display: none; }
        .status-badge {
            display: inline-block;
            padding: 5px 10px;
            border-radius: 20px;
            font-size: 0.9em;
            font-weight: bold;
        }
        .status-active { background: #10b981; color: white; }
        .status-idle { background: #6b7280; color: white; }
        .inline-btns { display:flex; gap:8px; }
        .hint { color: #6b7280; font-size: 0.95em; margin-top: 6px; }
        .error { color: #ef4444; margin-top: 6px; }
    </style>
</head>
<body>
    <div class="container">
        <h1>💪 Workout Form Trainer</h1>
        
        <div class="video-container">
            <img src="/video" id="videoFeed">
        </div>
        
        <div class="current-exercise">
            <h3 id="currentExercise">No Exercise Active</h3>
            <span class="status-badge status-idle" id="statusBadge">Idle</span>
        </div>
        
        <div class="queue-section">
            <h2>📋 Build Exercise Queue</h2>
            <div id="exerciseHint" class="hint hidden">No exercises found. Run the extraction cell in the notebook to generate .pkl reference files, then refresh this page.</div>
            <div id="exerciseError" class="error hidden"></div>
            <div class="form-grid-3">
                <select id="exerciseSelect" class="full-width">
                    <option value="">-- Select Exercise --</option>
                </select>
                <input type="number" id="sets" placeholder="Sets" min="1">
                <input type="number" id="reps" placeholder="Reps" min="1">
                <input type="number" id="weight" placeholder="Weight (kg)" step="0.1" min="0">
                <select id="targetMuscle">
                    <option value="">Target Muscle (optional)</option>
                    <option>Chest</option><option>Back</option><option>Legs</option><option>Arms</option><option>Shoulders</option><option>Core</option>
                </select>
                <select id="exerciseCategory">
                    <option value="">Category (optional)</option>
                    <option>Compound</option><option>Isolation</option>
                </select>
            </div>
            <div class="inline-btns">
                <button onclick="addToQueue()">➕ Add to Queue</button>
                <button onclick="clearQueue()" class="btn-secondary">🧹 Clear</button>
            </div>
            <div class="exercise-list" id="exerciseQueue"></div>
        </div>
        
        <div class="inline-btns">
            <button onclick="nextExercise()" class="btn-success" id="nextBtn">▶️ Start Next Exercise</button>
            <button onclick="stopExercise()" class="btn-danger hidden" id="stopBtn">⏹️ Stop Current Exercise</button>
        </div>
        
        <div class="queue-section">
            <h2>📊 Workout Details & Prediction</h2>
            <form id="predictionForm" onsubmit="submitPrediction(event)">
                <div class="form-grid">
                    <input type="number" id="age" placeholder="Age" required>
                    <select id="gender" required>
                        <option value="">Gender</option>
                        <option value="M">Male</option>
                        <option value="F">Female</option>
                    </select>
                    <input type="number" id="frequency" placeholder="Days/Week" required>
                    <input type="number" step="0.1" id="protein" placeholder="Protein (g/day)" required>
                    <input type="number" id="calories" placeholder="Calories/day" required>
                    <input type="number" step="0.1" id="sleep" placeholder="Sleep (hours)" required>
                    <select id="experience" required>
                        <option value="">Experience Level</option>
                        <option value="Beginner">Beginner</option>
                        <option value="Intermediate">Intermediate</option>
                        <option value="Advanced">Advanced</option>
                    </select>
                    <input type="number" step="0.1" id="currentSize" placeholder="Current Size (cm²)" required>
                    <input type="number" step="0.1" id="workoutYears" placeholder="Years Training" required>
                    <input type="number" id="timeMonths" placeholder="Time Horizon (months)" required class="full-width">
                </div>
                <button type="submit" class="btn-success">🔮 Predict Muscle Growth</button>
            </form>
            <div id="predictionResult" class="hidden"></div>
        </div>
    </div>
    
    <script>
        function show(el, flag) { el.classList.toggle('hidden', !flag); }
        function setText(el, text) { el.textContent = text; }
        
        function loadExercises() {
            fetch('/api/exercises')
                .then(r => r.json())
                .then(data => {
                    const select = document.getElementById('exerciseSelect');
                    const hint = document.getElementById('exerciseHint');
                    select.innerHTML = '<option value="">-- Select Exercise --</option>';
                    if (!data.exercises || data.exercises.length === 0) {
                        show(hint, true);
                    } else {
                        show(hint, false);
                        data.exercises.forEach(ex => {
                            select.innerHTML += `<option value="${ex}">${ex}</option>`;
                        });
                    }
                }).catch(() => {
                    const hint = document.getElementById('exerciseHint');
                    setText(hint, 'Failed to load exercises. Check server.');
                    show(hint, true);
                });
        }
        
        function addToQueue() {
            const ex = document.getElementById('exerciseSelect').value;
            const sets = parseInt(document.getElementById('sets').value || '0');
            const reps = parseInt(document.getElementById('reps').value || '0');
            const weight = parseFloat(document.getElementById('weight').value || '0');
            const target_muscle_group = document.getElementById('targetMuscle').value || null;
            const exercise_category = document.getElementById('exerciseCategory').value || null;
            const err = document.getElementById('exerciseError');
            show(err, false);
            if (!ex) { setText(err, 'Please select an exercise'); show(err, true); return; }
            if (!sets || !reps) { setText(err, 'Please provide sets and reps'); show(err, true); return; }
            
            fetch('/api/queue/add', {
                method: 'POST',
                headers: {'Content-Type': 'application/json'},
                body: JSON.stringify({
                    exercise_name: ex,
                    sets, reps, weight,
                    target_muscle_group, exercise_category
                })
            }).then(async (r) => {
                const data = await r.json().catch(() => ({}));
                if (!r.ok || (data && data.error)) {
                    setText(err, 'Add failed: ' + (data.error || r.statusText));
                    show(err, true);
                    return;
                }
                updateQueue();
                clearAddForm();
            }).catch(e => { setText(err, 'Add failed: ' + e); show(err, true); });
        }
        function clearAddForm() {
            ['sets','reps','weight','targetMuscle','exerciseCategory'].forEach(id => document.getElementById(id).value = '');
        }
        
        function clearQueue() {
            if (confirm('Clear all exercises from queue?')) {
                fetch('/api/queue/clear', {method: 'POST'}).then(updateQueue);
            }
        }
        function removeFromQueue(i) {
            fetch('/api/queue/remove', {
                method: 'POST',
                headers: {'Content-Type': 'application/json'},
                body: JSON.stringify({ index: i })
            }).then(updateQueue);
        }
        
        function nextExercise() {
            fetch('/api/exercise/next', {method: 'POST'})
                .then(r => r.json())
                .then(data => { if (data.error) alert(data.error); updateStatus(); });
        }
        function stopExercise() { fetch('/api/exercise/stop', {method: 'POST'}).then(updateStatus); }
        
        function updateQueue() {
            fetch('/api/queue')
                .then(r => r.json())
                .then(data => {
                    const container = document.getElementById('exerciseQueue');
                    if (!data.queue || data.queue.length === 0) {
                        container.innerHTML = '<p style="text-align:center;color:#6b7280;">Queue is empty</p>';
                    } else {
                        container.innerHTML = data.queue.map((item, i) => `
                            <div class="exercise-item ${i === data.current_index ? 'active' : ''}">
                                <div>
                                    <div><strong>${i+1}. ${item.exercise_name}</strong></div>
                                    <div class="meta">${item.sets} x ${item.reps} @ ${item.weight}kg
                                        ${item.target_muscle_group ? ' • ' + item.target_muscle_group : ''}
                                        ${item.exercise_category ? ' • ' + item.exercise_category : ''}
                                    </div>
                                </div>
                                <div class="inline-btns">
                                    ${i >= data.current_index ? `<button class="btn-danger" onclick="removeFromQueue(${i})">Remove</button>` : ''}
                                </div>
                            </div>`).join('');
                    }
                });
        }
        
        function updateStatus() {
            fetch('/api/status')
                .then(r => r.json())
                .then(data => {
                    const currentEx = document.getElementById('currentExercise');
                    const badge = document.getElementById('statusBadge');
                    const nextBtn = document.getElementById('nextBtn');
                    const stopBtn = document.getElementById('stopBtn');
                    if (data.is_active) {
                        currentEx.textContent = data.current_exercise || 'Active';
                        badge.textContent = 'Active';
                        badge.className = 'status-badge status-active';
                        nextBtn.classList.add('hidden');
                        stopBtn.classList.remove('hidden');
                    } else {
                        currentEx.textContent = 'No Exercise Active';
                        badge.textContent = 'Idle';
                        badge.className = 'status-badge status-idle';
                        nextBtn.classList.remove('hidden');
                        stopBtn.classList.add('hidden');
                    }
                    updateQueue();
                });
        }
        
        function submitPrediction(e) {
            e.preventDefault();
            const f = document.getElementById('predictionForm');
            const body = {
                age: parseInt(f.age.value),
                gender: f.gender.value,
                frequency: parseInt(f.frequency.value),
                protein: parseFloat(f.protein.value),
                calories: parseInt(f.calories.value),
                sleep: parseFloat(f.sleep.value),
                experience: f.experience.value,
                current_size_cm: parseFloat(f.currentSize.value),
                workout_time_years: parseFloat(f.workoutYears.value),
                time_months: parseInt(f.timeMonths.value)
            };
            fetch('/api/predict', { method:'POST', headers:{'Content-Type':'application/json'}, body: JSON.stringify(body) })
            .then(r => r.json())
            .then(result => {
                const container = document.getElementById('predictionResult');
                if (result.error) {
                    container.innerHTML = `<div style=\"color:#ef4444;\">Error: ${result.error}</div>`;
                } else {
                    let html = '<div class="prediction-result"><h3>🎯 Predicted Muscle Growth</h3>';
                    for (let [months, predictions] of Object.entries(result.result)) {
                        html += `<div class="muscle-growth"><strong>After ${months} months:</strong><ul>`;
                        for (let [muscle, growth] of Object.entries(predictions)) html += `<li>${muscle}: ${growth}</li>`;
                        html += '</ul></div>';
                    }
                    html += '</div>';
                    container.innerHTML = html;
                }
                container.classList.remove('hidden');
            }).catch(err => alert('Prediction failed: ' + err));
        }
        
        // Initialize
        loadExercises();
        updateStatus();
        setInterval(updateStatus, 2000);
    </script>
</body>
</html>
'''

@app.route('/')
def index():
    return render_template_string(HTML_TEMPLATE)

def gen():
    global current_frame
    while True:
        if current_frame is not None:
            yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + current_frame + b'\r\n')
        time.sleep(0.01)

@app.route('/video')
def video_feed():
    return Response(gen(), mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/api/exercises')
def api_exercises():
    return jsonify({'exercises': get_available_exercises()})

@app.route('/api/queue')
def api_queue():
    global exercise_queue, current_exercise_index
    return jsonify({
        'queue': exercise_queue,
        'current_index': current_exercise_index
    })

@app.route('/api/queue/add', methods=['POST'])
def api_queue_add():
    global exercise_queue
    data = request.json or {}
    ex = data.get('exercise_name')
    sets = data.get('sets')
    reps = data.get('reps')
    weight = data.get('weight', 0)
    target_muscle_group = data.get('target_muscle_group')
    exercise_category = data.get('exercise_category')
    if not ex or ex not in get_available_exercises():
        return jsonify({'error': 'Invalid exercise. Make sure you ran the extraction and selected from the dropdown.'}), 400
    if not isinstance(sets, int) or not isinstance(reps, int) or sets <= 0 or reps <= 0:
        return jsonify({'error': 'Provide valid integer sets and reps (>0).'}), 400
    try:
        weight = float(weight)
    except Exception:
        weight = 0.0
    exercise_queue.append({
        'exercise_name': ex,
        'sets': sets,
        'reps': reps,
        'weight': weight,
        'target_muscle_group': target_muscle_group,
        'exercise_category': exercise_category
    })
    return jsonify({'success': True, 'queue': exercise_queue})

@app.route('/api/queue/remove', methods=['POST'])
def api_queue_remove():
    global exercise_queue, current_exercise_index
    data = request.json or {}
    idx = data.get('index')
    if idx is None or not isinstance(idx, int) or idx < 0 or idx >= len(exercise_queue):
        return jsonify({'error': 'Invalid index'}), 400
    # Prevent removing already completed items
    if idx < current_exercise_index:
        return jsonify({'error': 'Cannot remove an already completed item'}), 400
    exercise_queue.pop(idx)
    return jsonify({'success': True, 'queue': exercise_queue})

@app.route('/api/queue/clear', methods=['POST'])
def api_queue_clear():
    global exercise_queue, current_exercise_index
    exercise_queue = []
    current_exercise_index = 0
    return jsonify({'success': True})

@app.route('/api/exercise/next', methods=['POST'])
def api_exercise_next():
    global exercise_queue, current_exercise_index, is_exercising, exercise_active
    if exercise_active:
        return jsonify({'error': 'Exercise already in progress'}), 400
    if not exercise_queue or current_exercise_index >= len(exercise_queue):
        return jsonify({'error': 'No exercises in queue'}), 400
    item = exercise_queue[current_exercise_index]
    exercise_name = item['exercise_name']
    current_exercise_index += 1
    def run_exercise():
        global is_exercising
        is_exercising = True
        perform_exercise_and_predict(exercise_name)
        is_exercising = False
    thread = threading.Thread(target=run_exercise, daemon=True)
    thread.start()
    return jsonify({'success': True, 'exercise': exercise_name})

@app.route('/api/exercise/stop', methods=['POST'])
def api_exercise_stop():
    global stop_exercise_flag
    stop_exercise_flag = True
    return jsonify({'success': True})

@app.route('/api/status')
def api_status():
    global exercise_queue, current_exercise_index, exercise_active
    current_ex = None
    if exercise_active and current_exercise_index > 0 and current_exercise_index <= len(exercise_queue):
        current_ex = exercise_queue[current_exercise_index - 1]['exercise_name']
    return jsonify({
        'is_active': exercise_active,
        'current_exercise': current_ex,
        'queue_length': len(exercise_queue),
        'current_index': current_exercise_index
    })

@app.route('/api/predict', methods=['POST'])
def api_predict():
    global exercise_queue, current_exercise_index
    data = request.json or {}
    # Build completed exercise list with details
    completed = exercise_queue[:current_exercise_index] if current_exercise_index > 0 else []
    if not completed:
        return jsonify({'error': 'No completed exercises yet. Start and finish at least one.'}), 400
    payload = {
        'age': data.get('age'),
        'gender': data.get('gender'),
        'exercises': completed,
        'frequency': data.get('frequency'),
        'protein': data.get('protein'),
        'calories': data.get('calories'),
        'sleep': data.get('sleep'),
        'experience': data.get('experience'),
        'current_size_cm': data.get('current_size_cm'),
        'workout_time_years': data.get('workout_time_years'),
        'time_months': data.get('time_months')
    }
    try:
        response = requests.post(BULK_API_URL, json=payload, timeout=20)
        if response.status_code == 200:
            return jsonify(response.json())
        else:
            return jsonify({'error': f'API error: {response.status_code}', 'details': response.text}), 500
    except Exception as e:
        return jsonify({'error': str(e)}), 500

def run_server():
    app.run(host='0.0.0.0', port=5000, threaded=True, use_reloader=False)

thread = threading.Thread(target=run_server, daemon=True)
thread.start()
print("🚀 Streaming server running on http://[YOUR-PC-IP]:5000")
print("📱 Open this URL on your phone to access the workout interface!")

🚀 Streaming server running on http://[YOUR-PC-IP]:5000 * Serving Flask app '__main__'

📱 Open this URL on your phone to access the workout interface!
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


## How to Use (Per-Exercise Details)

1. Run the extraction cell to generate `.pkl` files from videos.
2. Start the Flask server (the cell above).
3. Open `http://[YOUR-PC-IP]:5000` on your phone.
4. In "Build Exercise Queue":
   - Pick an exercise, enter Sets, Reps, Weight, and optionally Target Muscle + Category.
   - Click "Add to Queue". Repeat for your full workout.
5. Tap "Start Next Exercise" to begin the next item in your queue. The stream updates live.
6. After completing at least one exercise, fill in the Workout Details including Time Horizon (months) and tap "Predict Muscle Growth".
7. Your prediction results show below the form.

Notes:
- You can remove any upcoming exercise from the queue (not the ones already completed).
- Predictions use the list of completed exercises with their individual sets/reps/weights, matching the model expected schema.
- Ensure the growth predictor service from `32Fit/bulk.ipynb` is running on `http://localhost:3001/bulk`. Update `BULK_API_URL` if different.